In [1]:
import torch

In [2]:
!nvidia-smi

Sat Sep 21 11:13:54 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.44                 Driver Version: 552.44         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P3             12W /   30W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import os 
import requests 

pdf_path = "human-nutrition-text.pdf"

if not os.path.exists(pdf_path):
    print(f"[INFO] File doesn't exist downloading")

    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"
    filename = pdf_path
    
    response = requests.get(url)

    if response.status_code == 200:
        with open(filename,'wb') as file:
            file.write(response.content)
        
        print(f"[INFO] the file has been downloaded and saved as {filename}")
    
    else:
        print(f"[INFO] failed to download the file.Status code : {response.status_code}" )

else:
    print("File exists")

File exists


In [12]:
import fitz 
from tqdm import tqdm

def text_formatter(text : str) -> str:
    """Performs minor formatiing on text"""
    cleaned_text = text.replace("\n", " ").strip()
    
    return cleaned_text

def open_and_read_pdf(pdf_path : str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number , page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({"Page number" : page_number - 41,
                                "page_char_count" : len(text),
                                "page_word_count" : len(text.split(" ")),
                                "page_sentence_count" : len(text.split(" ")),
                                "page_sentence_count_raw" : len(text.split(". ")),
                                "page_token_counts" : len(text)/4,
                                "text" : text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]
        

1208it [00:01, 733.65it/s]


[{'Page number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_counts': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'Page number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_counts': 0.0,
  'text': ''}]

In [13]:
import random 
random.sample(pages_and_texts,k=3)

[{'Page number': 1058,
  'page_char_count': 244,
  'page_word_count': 49,
  'page_sentence_count': 49,
  'page_sentence_count_raw': 3,
  'page_token_counts': 61.0,
  'text': 'iron, omega-3 fatty acids, zinc, and vitamin B12. Also, if people who  follow these diets do not plan out their meals, they may gravitate  toward foods high in fats.  Table 18.1 The Pros and Cons of Seven Popular Diets  1058  |  Comparing Diets'},
 {'Page number': 866,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_counts': 0.0,
  'text': ''},
 {'Page number': 223,
  'page_char_count': 2161,
  'page_word_count': 363,
  'page_sentence_count': 363,
  'page_sentence_count_raw': 18,
  'page_token_counts': 540.25,
  'text': 'drinking sports drinks instead of plain water enhances endurance  or performance in individuals exercising less than one hour and at  low to moderate intensities. A well-concocted sports drink contains  sugar, water, and sod

In [14]:
import pandas as pd 
df = pd.DataFrame(pages_and_texts)
df.head()

,Page number,page_char_count,page_word_count,page_sentence_count,page_sentence_count_raw,page_token_counts,text
0,-41,29,4,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,1,0.00,
2,-39,320,54,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,147,147,3,199.25,Contents Preface University of Hawai‘i at Mā...


In [15]:
df.describe().round(2)

,Page number,page_char_count,page_word_count,page_sentence_count,page_sentence_count_raw,page_token_counts
count,1208.00,1208.00,1208.00,1208.00,1208.00,1208.00
mean,562.50,1148.00,199.50,199.50,10.52,287.00
std,348.86,560.38,95.83,95.83,6.55,140.10
min,-41.00,0.00,1.00,1.00,1.00,0.00
25%,260.75,762.00,134.00,134.00,5.00,190.50
50%,562.50,1231.50,216.00,216.00,10.00,307.88
75%,864.25,1603.50,272.00,272.00,15.00,400.88
max,1166.00,2308.00,430.00,430.00,39.00,577.00
